# 🎬 AI Video Generator - ASYNC VERSION (No Timeouts!)

## ✅ ZERO CONFIGURATION NEEDED!

**How it works:**
1. Backend sends job → Gets job_id instantly ⚡
2. Colab processes in background (keeps session alive) 🔄
3. Backend polls status every 10s 📊
4. When done, downloads video ✅
5. **NO NGROK TIMEOUTS!** 🎉

**Just run cells 1→2→3→4→5→6→7 in order!**

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 1: INSTALL EVERYTHING
# ═══════════════════════════════════════════════════════════════════════════════
import sys
print(f"Python: {sys.version}\n")

if sys.version_info >= (3, 12):
    print("Installing Python 3.10...")
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    print("⚠️  Runtime restarting - Run this cell again after restart!")
else:
    print("Installing packages...\n")
    !apt-get update -qq && apt-get install -y -qq ffmpeg
    !pip install -q flask flask-cors pyngrok requests torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
    !pip install -q TTS soundfile numpy scipy diffusers transformers accelerate safetensors pillow opencv-python-headless
    print("\n✅ All installed! Run Cell 2")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 2: IMPORTS (Auto-install if missing)
# ═══════════════════════════════════════════════════════════════════════════════
import sys, subprocess

def check(pkg, imp=None):
    try: __import__(imp or pkg.replace('-','_'))
    except: subprocess.run([sys.executable,'-m','pip','install','-q',pkg]); print(f"Installed {pkg}")

check('flask'); check('flask-cors','flask_cors'); check('pyngrok'); check('torch'); check('pillow','PIL')

import torch, json, io, gc, time, base64
from pathlib import Path
from PIL import Image
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
from threading import Thread
from datetime import datetime

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"GPU: {torch.cuda.get_device_name(0) if device=='cuda' else 'None'}")

output_dir = Path('/content/outputs')
output_dir.mkdir(exist_ok=True)
print("✅ Ready")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 3: IMAGE MODEL (Auto-install)
# ═══════════════════════════════════════════════════════════════════════════════
import sys, subprocess
try: from diffusers import StableDiffusionXLPipeline
except: subprocess.run([sys.executable,'-m','pip','install','-q','diffusers','transformers','accelerate']); from diffusers import StableDiffusionXLPipeline

img_pipeline = None
STYLES = {"cinematic":{"p":"cinematic, movie quality","n":"low quality"},"anime":{"p":"anime style, manga","n":"photorealistic"},"horror":{"p":"dark, creepy, terrifying","n":"bright, cheerful"}}

def load_img():
    global img_pipeline
    if not img_pipeline:
        print("Loading DreamShaper XL...")
        img_pipeline = StableDiffusionXLPipeline.from_pretrained("Lykon/dreamshaper-xl-1-0",torch_dtype=torch.float16,variant="fp16").to(device)
        img_pipeline.enable_xformers_memory_efficient_attention()
    return img_pipeline

def gen_img(prompt, style="cinematic"):
    p = load_img()
    s = STYLES.get(style, STYLES["cinematic"])
    return p(prompt=f"{prompt}, {s['p']}", negative_prompt=s['n'], num_inference_steps=25, height=864, width=1536).images[0]

print("✅ Image ready")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 4: VOICE MAPPING + TTS
# ═══════════════════════════════════════════════════════════════════════════════
VOICES = {'guy':'p226','adam':'p226','brian':'p227','aria':'p229','sarah':'p231','nicole':'p233','jenny':'p228','emma':'p230'}
def get_voice(v): return VOICES.get(v,'p226')

import sys, subprocess
try: import TTS
except:
    if sys.version_info >= (3, 12):
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'coqui-tts', 'soundfile'], check=True)
    else:
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'TTS', 'soundfile'], check=True)

tts_model = None

def load_tts():
    global tts_model
    if not tts_model:
        from TTS.api import TTS as TTSModel
        print("Loading TTS model...")
        tts_model = TTSModel("tts_models/en/vctk/vits").to(device)
    return tts_model

def gen_voice(text, voice, path):
    load_tts().tts_to_file(text=text, speaker=get_voice(voice), file_path=path)
    return path

print("✅ TTS ready")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 5: ASYNC JOB QUEUE SYSTEM
# ═══════════════════════════════════════════════════════════════════════════════

# Job storage: {job_id: {status, progress, video_path, error, created_at}}
jobs = {}

def process_video_job(job_id, data):
    """Process video generation in background thread"""
    try:
        jobs[job_id]['status'] = 'processing'
        jobs[job_id]['progress'] = 0
        
        wd = output_dir / job_id
        wd.mkdir(exist_ok=True)
        
        # Generate images
        image_prompts = data.get('image_prompts', [])
        style = data.get('style', 'cinematic')
        
        print(f"[{job_id}] Generating {len(image_prompts)} images...")
        for i, prompt in enumerate(image_prompts):
            img = gen_img(prompt, style)
            img.save(wd / f"{i:04d}.png")
            jobs[job_id]['progress'] = int((i + 1) / len(image_prompts) * 40)  # 0-40%
            print(f"[{job_id}] Image {i+1}/{len(image_prompts)} done ({jobs[job_id]['progress']}%)")
        
        # Generate voice
        print(f"[{job_id}] Generating voice...")
        jobs[job_id]['progress'] = 50
        ap = wd / "voice.wav"
        gen_voice(data.get('script', ''), data.get('voice', 'aria'), str(ap))
        jobs[job_id]['progress'] = 70
        
        # Compile video
        print(f"[{job_id}] Compiling video...")
        jobs[job_id]['progress'] = 80
        op = wd / "final.mp4"
        
        subprocess.run([
            'ffmpeg', '-y', '-hwaccel', 'cuda',
            '-framerate', '30',
            '-pattern_type', 'glob',
            '-i', str(wd / '*.png'),
            '-i', str(ap),
            '-c:v', 'h264_nvenc',
            '-b:v', '10M',
            '-c:a', 'aac',
            '-s', '1920x1080',
            str(op)
        ], check=True, capture_output=True)
        
        # Complete
        jobs[job_id]['status'] = 'completed'
        jobs[job_id]['progress'] = 100
        jobs[job_id]['video_path'] = str(op)
        print(f"[{job_id}] ✅ COMPLETED!")
        
    except Exception as e:
        jobs[job_id]['status'] = 'failed'
        jobs[job_id]['error'] = str(e)
        print(f"[{job_id}] ❌ FAILED: {e}")

print("✅ Job queue ready")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 6: FLASK SERVER WITH ASYNC ENDPOINTS
# ═══════════════════════════════════════════════════════════════════════════════
app = Flask(__name__)
CORS(app)

@app.route('/health')
def health():
    return jsonify({"ok": True, "gpu": device=='cuda', "active_jobs": len([j for j in jobs.values() if j['status'] == 'processing'])})

@app.route('/submit_job', methods=['POST'])
def submit_job():
    """Submit video generation job - returns job_id immediately"""
    try:
        data = request.get_json(force=True)
        
        import uuid
        job_id = str(uuid.uuid4())
        
        # Create job entry
        jobs[job_id] = {
            'status': 'queued',
            'progress': 0,
            'video_path': None,
            'error': None,
            'created_at': datetime.now().isoformat()
        }
        
        # Start background processing
        Thread(target=process_video_job, args=(job_id, data), daemon=True).start()
        
        print(f"\n🚀 Job {job_id} submitted!")
        
        return jsonify({
            "success": True,
            "job_id": job_id,
            "message": "Job queued - check /job_status/<job_id> for progress"
        })
        
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/job_status/<job_id>')
def job_status(job_id):
    """Check job status and progress"""
    if job_id not in jobs:
        return jsonify({"error": "Job not found"}), 404
    
    job = jobs[job_id]
    return jsonify({
        "job_id": job_id,
        "status": job['status'],
        "progress": job['progress'],
        "error": job.get('error'),
        "created_at": job.get('created_at')
    })

@app.route('/download/<job_id>')
def download(job_id):
    """Download completed video"""
    if job_id not in jobs:
        return jsonify({"error": "Job not found"}), 404
    
    job = jobs[job_id]
    
    if job['status'] != 'completed':
        return jsonify({"error": f"Job not completed (status: {job['status']})"}), 400
    
    video_path = Path(job['video_path'])
    if not video_path.exists():
        return jsonify({"error": "Video file not found"}), 404
    
    return send_file(str(video_path), mimetype='video/mp4')

print("✅ Server ready with async endpoints")
print("   POST /submit_job → Get job_id instantly")
print("   GET /job_status/<job_id> → Check progress")
print("   GET /download/<job_id> → Download video")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 7: START SERVER
# ═══════════════════════════════════════════════════════════════════════════════

YOUR_NGROK_TOKEN = "35HuufK0IT26RER84mcvIbRjrog_7grjZvuDXtRPYL5hWLNCK"

# Start ngrok tunnel
ngrok.set_auth_token(YOUR_NGROK_TOKEN)
url = ngrok.connect(5001)

print("\n" + "="*80)
print(f"🌐 ASYNC SERVER RUNNING AT: {url}")
print("="*80)
print(f"Add this to your backend config/__init__.py:")
print(f"COLAB_SERVER_URL = '{url}'")
print("="*80)
print("\n📋 API Endpoints:")
print(f"   Health: {url}/health")
print(f"   Submit Job: {url}/submit_job (POST)")
print(f"   Job Status: {url}/job_status/<job_id> (GET)")
print(f"   Download: {url}/download/<job_id> (GET)")
print("\n🔥 NO MORE TIMEOUT ISSUES!")
print("   Jobs process in background while ngrok stays alive!")
print("="*80 + "\n")

# Start Flask server
try:
    app.run(host='0.0.0.0', port=5001, debug=False, use_reloader=False)
except KeyboardInterrupt:
    print("\n✅ Server stopped")